# Tracy Reinhold: Global Security and Resilience Authority

### "You can get ahead of threats by being informed and having the technological capabilities to actually take action on that intelligence once you’ve acquired it. So if you’re consistently in a reactive mode, you will eventually fail. By proactively reducing risk and vulnerability, we can create an environment that allows a company to be successful." —via [Security Magazine](https://www.securitymagazine.com/articles/96603-5-minutes-with-tracy-reinhold-critical-event-management-for-enterprise-resilience)

### We consider the arc of a critical event and the way Tracy considers the future of predictive approaches with AI.

| Before the 💥 Boom | During the 💥 Boom | After the 💥 Boom |
|---|---|---|

Note that there are three phases to a critical event: before, during, and after. The critical event is often referred to as "💥 boom" or "💥 bang."

Instructions for getting cozy with this AI recipe are on the [GitHub page's README](https://aka.ms/CAIK-repo). 

# 🧑‍🍳 Recipe for AI-driven Operational Resilience

- ~500 tokens from COMPLETION Pre-trained Foundation Model
- ~1000 tokens from EMBEDDING Pre-trained Foundation Model
 
First, register a variety of pieces of data into the memory store with the EMBEDDING model's help.

Then we look for any patterns that could be troubling.

Take all those patterns and generate a quick report.

> [!IMPORTANT]
> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Gather the core tools 🧰

When running the following cell, if asked to "select your kernel" (note this will be referring to the Jupyter notebook's kernel and not Semantic Kernel) then choose `.NET Interactive` from the available menu options.

In [ ]:
// Load some helper functions, e.g. to load values from settings.json
#!import ../config/Settings.cs 

#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta3"
#r "nuget: System.Linq.Async, 6.0.1"

# Fire up a kernel with TWO models 🔥

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var memoryBuilder = new MemoryBuilder();
var kernelBuilder = new KernelBuilder();

if (useAzureOpenAI)
{
    memoryBuilder.WithAzureTextEmbeddingGenerationService("text-embedding-ada-002", azureEndpoint, apiKey);
    kernelBuilder.WithAzureChatCompletionService(model, azureEndpoint, apiKey);
}
else
{
    memoryBuilder.WithOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
    kernelBuilder.WithOpenAIChatCompletionService(model, apiKey, orgId);
}

memoryBuilder.WithMemoryStore(new VolatileMemoryStore());

var kernel = kernelBuilder.Build();
var memory = memoryBuilder.Build();

# Generate embeddings for later 🧲 gathering

In [ ]:
const string MemoryCollectionName = "kevinQuotes";

// Kevin quoted by his fans on GoodReads: https://www.goodreads.com/author/quotes/21599683.Kevin_G_Bethune

await memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign1", 
    text: "I believe it’s very important to be more up-front about the hard realities we will face as innovators and still push ourselves forward with a sense of optimism and courage.");
await memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign2", 
    text: "The team needs to learn how to work together, align around the right imperatives, and develop their unique chemistry.");
await memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign3", 
    text: "The constellation of insights must line up in a compelling way, and usually those insights need to come from different vantage points to ensure an idea has a shot of making it into a well-timed, successful innovation that is desirable, viable, and technically feasible.");
await memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign4", 
    text: "We should leverage uncertainty for the powerful, creativity-inducing variable that it is.");
await memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign5", 
    text: "The quest for innovation against newly emerging realities requires us to bring disparate teams closer together and to create more bandwidth for actually collaborating with one another.");
await memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign6", 
    text: "Design still fights the perception of being the very last step in the value stream.");
await memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign7", 
    text: "Thanks to design thinking, most organizations are beginning to see the value of embracing design earlier in their processes.");
await memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign8", 
    text: "Moving on to incubation, as we build out our designed experiences, the designer should no longer (and can no longer) work in isolation from other disciplines.");
await memory.SaveInformationAsync(MemoryCollectionName, id: "ReimaginingDesign9", 
    text: "Design will play an important role in stringing the collective team’s efforts together in a cohesive way to optimize impact in the marketplace.");

# Experiment 🧪 with what you think is relevant and important 

In [ ]:
var questions = new[]
{
    "how to make a pizza",
    "challenges of businesses"
};

foreach (var q in questions)
{
    var response = await kernel.Memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
    Console.WriteLine($"{q}\n> {response?.Metadata.Text}");
}

# Plate the partially finished meal 🍽️

In [ ]:
using Microsoft.AspNetCore.Html;
using Markdig;

String foundInMemory = "";

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q).FirstOrDefaultAsync();
   foundInMemory += $"## {q}\n> {response?.Metadata.Text}\n\n";
}

var myPipeline = new MarkdownPipelineBuilder().UseAdvancedExtensions().Build();
var myResult2HTML = Markdown.ToHtml(foundInMemory, myPipeline);
var myHTMLContent = new HtmlContentBuilder();

myHTMLContent.AppendHtml(myResult2HTML);
myHTMLContent

# Now let's use a ✨ COMPLETION model that's grounded

In [ ]:
var pluginsSKDirectory = "plugins-sk";

var mnFunctions = kernel.ImportSemanticSkillFromDirectory(pluginsSKDirectory, "Mentor");

var result = await kernel.RunAsync(@"
What should I become?", mnFunctions["CareerAdviceGeneric"]);
var resultString = result.GetValue<string>();
Console.WriteLine(resultString);


That was totally made up. Right? Let's now get the model grounded in things that Kevin actually believes in :+).

In [ ]:
using Microsoft.SemanticKernel.Orchestration;

var myContext = new ContextVariables(); 

myContext.Set("input", "What should I become?"); 
myContext.Set("beliefs", "I believe it’s very important to be more up-front about the hard realities we will face as innovators and still push ourselves forward with a sense of optimism and courage."); 

var myResult = await kernel.RunAsync(myContext,mnFunctions["CareerAdviceGrounded"]);
var resultString = result.GetValue<string>();

Console.WriteLine(myResult);


# Let's look at the science 📐

In [ ]:
using System.Text.Json;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI;

var usageresult = myResult.FunctionResults.SelectMany(l => l.GetModelResults() ?? Enumerable.Empty<ModelResult>()).LastOrDefault()?.GetOpenAIChatResult()?.Usage;
var injson = JsonSerializer.Serialize(usageresult, new JsonSerializerOptions { WriteIndented = true });
Console.WriteLine($"🧱 The MODEL used:\n\n> {model}\n");
Console.WriteLine($"💸 How many TOKENS were used?\n\n{injson}\n");
String functionName = "CareerAdviceGrounded";

var reqjson = JsonSerializer.Serialize(mnFunctions[functionName].RequestSettings, new JsonSerializerOptions { WriteIndented = true });
var doc = JsonDocument.Parse(reqjson);
double temperatureValue = doc.RootElement.GetProperty("temperature").GetDouble();
double topPValue = doc.RootElement.GetProperty("top_p").GetDouble();

Console.WriteLine($"🌡️ What TEMPERATURE setting was used?\n\n> {temperatureValue} (0 is 🥶 ; 1 is 🥵)\n");
Console.WriteLine($"🙊 What TOP P setting was used?\n\n> {topPValue} (0 is 😑 ; 1 is 🤯)");